## Описание проекта

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».  
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

## Знакомство с данными

Импортируем библиотеки

In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

Загружаем наш датасет

In [3]:
df = pd.read_csv('/Users/sirena0789/Downloads/users_behavior.csv')

Смотрим первые 5 строк и названия колонок

In [4]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Изучаем информацию о датасете

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**Вывод:**  
Всего имеется 3214 строк для обучения и теста.  
К признакам тут можно отнести количество звонков, суммарную длительность звонков в минутах,  количество sms-сообщений, израсходованный интернет-трафик в Мб, а вот каким тарифом пользовался в течение месяца абонент - целевой признак.

## Разбитие данных на выборки

Рарзбиваем нашу выборку на обучающую и "другую" в соотношении 60/40

In [6]:
df_train, df_other = train_test_split(df, test_size=0.4, random_state=12345)

Разбиваем выборки на признаки и целевые признаки

In [7]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_other = df_other.drop(['is_ultra'], axis=1)
target_other = df_other['is_ultra']

Рарзбиваем "другую" выборку на тестовую и валидационную в соотношении 50/50

In [8]:
features_valid, features_test, target_valid, target_test = train_test_split(
    features_other, target_other, test_size=0.5, random_state=12345)

In [20]:
print(features_valid.shape)
print(features_test.shape)
print(features_train.shape)

(643, 4)
(643, 4)
(1928, 4)


**Вывод:**  
Выборки разбиты на тестовую, обучающую и валидационную. Последние две будем использовать в третьем задании, а первую в четвертом.

## Исследование моделей

Модель случайный лес

In [9]:
model_forest = None
result_forest = 0
for est in range(1, 11):
    for depth in range(1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) # обучим модель с заданным количеством деревьев
        model.fit(features_train, target_train) # обучим модель на тренировочной выборке
        result = model.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
        if result > result_forest:
            model_forest = model# сохраним наилучшую модель
            result_forest = result#  сохраним наилучшее значение метрики accuracy на валидационных данных
            best_depth = depth
            best_est = est
print( " Глубина дерева:", best_depth, '\n', "Количество деревьев:", best_est, '\n', "Accuracy модели на валидационной выборке:", result_forest)

 Глубина дерева: 8 
 Количество деревьев: 8 
 Accuracy модели на валидационной выборке: 0.80248833592535


Модель логистической регрессии

In [10]:
model_logistic = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 
model_logistic.fit(features_train, target_train) 
model_logistic.predict(features_valid)
result_logistic = model_logistic.score(features_valid, target_valid)
print("Accuracy модели на валидационной выборке:", result_logistic)

Accuracy модели на валидационной выборке: 0.7107309486780715


Модель дерева решений 

In [11]:
model_tree = None
result_tree = 0
for depth in range(1,6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = model.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
    if result > result_tree:
        model_tree = model# сохраним наилучшую модель
        result_tree = result#  сохраним наилучшее значение метрики accuracy на валидационных данных
        best_depth = depth

print(" Глубина дерева:", best_depth, '\n', "Accuracy наилучшей модели на валидационной выборке:", result_tree)

 Глубина дерева: 3 
 Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692


**Вывод:**  
Лучшим образом себя показала модель случайного леса с гиперпараметрами "Глубина дерева: 8, Количество деревьев: 8". Ее и будем использовать для тестовой выборки.

## Проверка модели на тестовой выборке

Модель случайный лес

In [13]:
model = RandomForestClassifier(random_state=12345, n_estimators=8, max_depth=8)
model.fit(features_train, target_train)
accuracy_1 = model.score(features_test, target_test)
print("Качество:", accuracy_1)

Качество: 0.7962674961119751


## Проверка модели на адекватность

In [14]:
dmb = DummyClassifier(strategy="stratified" ,random_state=12345)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Качество прогноза стратегии stratified :", acc)
print("Качество прогноза модели случаного леса :", accuracy_1)

Качество прогноза стратегии stratified : 0.536547433903577
Качество прогноза модели случаного леса : 0.7962674961119751


In [15]:
dmb = DummyClassifier(strategy='uniform', random_state=12345)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Качество прогноза стратегии uniform :", acc)
print("Качество прогноза модели случаного леса :", accuracy_1)

Качество прогноза стратегии uniform : 0.4821150855365474
Качество прогноза модели случаного леса : 0.7962674961119751


In [16]:
dmb = DummyClassifier(strategy='most_frequent', random_state=12345)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Качество прогноза стратегии most_frequent :", acc)
print("Качество прогноза модели случаного леса :", accuracy_1)

Качество прогноза стратегии most_frequent : 0.6842923794712286
Качество прогноза модели случаного леса : 0.7962674961119751


In [17]:
dmb = DummyClassifier(strategy='prior', random_state=12345)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Качество прогноза стратегии prior :", acc)
print("Качество прогноза модели случаного леса :", accuracy_1)

Качество прогноза стратегии prior : 0.6842923794712286
Качество прогноза модели случаного леса : 0.7962674961119751


**Вывод:**  
Наша модель работает лучше классификатора. Следовательно, можно полагать, что модель работает адекватно.

## Вывод

Проверены модели: деревья решений, логистическая регрессия, случайный лес.
Наилучший результат дала модель случайного леса.  
Гиперпараметры: n_estimators=8, max_depth=8.  
Качество: 0.7962674961119751.  
Также модель прошла проверку на адекватность.